## Domestic Flight Delay Records
- **Analysis performed by :-** Russel Anthony Reynold Chandanshiv

---

- Copyright (c) 2025 Russel Anthony Chandanshiv
- Licensed under the MIT License
  
---

### Introduction

The data set ‘Domestic Flight Delay Records’ contains comprehensive data about domestic flights in the United States, such as flight dates, airtime, flight distance, scheduled departure/arrival times and departure and arrival delays.</br>
</br>
The Data Set contains **`7 Features` and `1,000,000 Observations`**. Each observation recorded in the Data Set represents a single flight.
</br>
</br> **Features**
- **`FL_DATE`** - Flight Date.
- **`DEP_DELAY`** - Departure delay (in minutes). `[Negative : Early Departure, Positive : Flight Delay, Zero : In-time]`
- **`ARR_DELAY`** - Arrival delay (in minutes). `[Negative : Early Arrival, Positive : Flight Delay, Zero : In-time]`
- **`AIR_TIME`** - Total Flight Duration (in Air).
- **`DISTANCE`** - Total Flight Distance (in Miles).
- **`DEP_TIME`** - Scheduled / Actual Departure Time.
- **`ARR_TIME`** - Scheduled / Actual Arrival Time.

**Libraries and Dependencies**
- PySpark

**Assumptions**
- `FL_DATE` is in MM/DD/YYY format
- `DEP_TIME` and `ARR_TIME` represent 24-hour clock time. HH/MM format.
- `DEP_DELAY` and `ARR_DELAY` and `AIR_TIME` measured in Minutes.
- `DISTANCE` measured in Miles.
- Each record represents one unique flight details.
- Negative values in `DEP_DELAY` and `ARR_DELAY` represent Early Departure / Early Arrival. Positive represents delay. Zero represents On-time.

**WARNING:** Do not re-run this notebook without installing all required libraries and dependencie
 

**Note:** The dataset used in this analysis is not publicly shared to ensure compliance with copyright and data ownership regulations. This notebook contains only the analytical workflow, methodologies, and visualizations derived from the dataset. All analysis and conclusions are original and created for educational and research purposes.

In [1]:
# Import required libraries
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, LongType, DateType, FloatType, DoubleType, TimestampType, DateType
from pyspark.sql.functions import col, trim, to_date

# --------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
# Initializing Spark session and Confirming if the Spark Session actually exists.

try:
    spark = (
             SparkSession.builder 
            .appName("DomesticFlightAnalysis")
            .getOrCreate()
)
except Exception:
    print ("SparkSession is not currently Active. This is due to {Exception}.")
else:
    if isinstance(spark, SparkSession) and "spark" in locals():
        print ("SparkSession is currently Active.")  


SparkSession is currently Active.


In [2]:
# Reading the Domestic Flight Dataset
domestic_flight_df = spark.read.csv ("Flight Dataset - CSV(in).csv", inferSchema = True, header = True)

# Caching the Dataset for faster computations
domestic_flight_df.cache()

# Calling an Action after caching (Viewing first 10 Rows)
print ("The first few rows of the PySpark Domestic Flight DataFrame as follows:\n")
domestic_flight_df.show(10)


The first few rows of the PySpark Domestic Flight DataFrame as follows:

+---------+---------+---------+--------+--------+---------+---------+
|  FL_DATE|DEP_DELAY|ARR_DELAY|AIR_TIME|DISTANCE| DEP_TIME| ARR_TIME|
+---------+---------+---------+--------+--------+---------+---------+
| 1/1/2006|        5|       19|     350|    2475| 9.083333|12.483334|
| 1/2/2006|      167|      216|     343|    2475|11.783334|15.766666|
| 1/3/2006|       -7|       -2|     344|    2475| 8.883333|12.133333|
| 1/4/2006|       -5|      -13|     331|    2475| 8.916667|    11.95|
| 1/5/2006|       -3|      -17|     321|    2475|     8.95|11.883333|
| 1/6/2006|       -4|      -32|     320|    2475| 8.933333|11.633333|
| 1/8/2006|       -3|       -2|     346|    2475|     8.95|12.133333|
| 1/9/2006|        3|        0|     334|    2475|     9.05|12.166667|
|1/10/2006|       -7|      -21|     334|    2475| 8.883333|11.816667|
|1/11/2006|        8|      -10|     321|    2475| 9.133333|     12.0|
+---------+------

In [3]:
# Print the schema of the DataFrame
# --------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
print ("The Schema of the PySpark Domestic Flight DataFrame is as follows:\n")
domestic_flight_df.printSchema()


The Schema of the PySpark Domestic Flight DataFrame is as follows:

root
 |-- FL_DATE: string (nullable = true)
 |-- DEP_DELAY: integer (nullable = true)
 |-- ARR_DELAY: integer (nullable = true)
 |-- AIR_TIME: integer (nullable = true)
 |-- DISTANCE: integer (nullable = true)
 |-- DEP_TIME: double (nullable = true)
 |-- ARR_TIME: double (nullable = true)



In [4]:
# Converting FL_DATE to suitable Date Format
# --------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
domestic_flight_df = domestic_flight_df.withColumn("FL_DATE", to_date(col("FL_DATE"), "M/d/yyyy"))

print ("The Schema of the PySpark Domestic Flight DataFrame (after conversion) is as follows:\n")
domestic_flight_df.printSchema()


The Schema of the PySpark Domestic Flight DataFrame (after conversion) is as follows:

root
 |-- FL_DATE: date (nullable = true)
 |-- DEP_DELAY: integer (nullable = true)
 |-- ARR_DELAY: integer (nullable = true)
 |-- AIR_TIME: integer (nullable = true)
 |-- DISTANCE: integer (nullable = true)
 |-- DEP_TIME: double (nullable = true)
 |-- ARR_TIME: double (nullable = true)



In [5]:
print (f"The total number of rows in our PySpark DataSet : {domestic_flight_df.count():,} Rows / Flights / Observations.")


The total number of rows in our PySpark DataSet : 1,000,000 Rows / Flights / Observations.


In [6]:
# Dropping Duplicates (If any)
domestic_flight_df = domestic_flight_df.dropDuplicates()

# Caching the Dataset for faster computations
domestic_flight_df.cache()

# After dropping duplicates (if any)
print (f"The Total number of rows in our PySpark DataSet (After dropping duplicates) : {domestic_flight_df.count():,} Rows / Flights / Observations.")


The Total number of rows in our PySpark DataSet (After dropping duplicates) : 999,994 Rows / Flights / Observations.


In [7]:
# Checking the Total Number of Missing Values in Our Dataset
# --------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
# Dropping the View if it exists
spark.sql("DROP VIEW IF EXISTS missing_value_view")

# --------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
# Creating a Temporary View
domestic_flight_df.createTempView("missing_value_view")

# --------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
# Check for missing values
number_of_missing_vals = spark.sql("""
                                    SELECT COUNT(*) AS `Number of Missing Values`
                                    FROM missing_value_view  
                                    WHERE 
                                    FL_DATE IS NULL 
                                    OR 
                                    DEP_DELAY IS NULL
                                    OR
                                    ARR_DELAY IS NULL
                                    OR
                                    AIR_TIME IS NULL
                                    OR
                                    DISTANCE IS NULL
                                    OR
                                    DEP_TIME IS NULL
                                    OR
                                    ARR_TIME IS NULL                                    
""")
# --------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
# Let's view the count of missing values
number_of_missing_vals.show()


+------------------------+
|Number of Missing Values|
+------------------------+
|                       0|
+------------------------+



---

#### Count Flights Arriving Earlier Than Expected

In [8]:
def early_arrivals(dataframe):
    """
    Docstring: 
              Take's a PySpark Dataframe as input.
              Checks how many flights arrived earlier than expected.
              Return a PySpark Dataframe with the Number of flights which arrived earlier than expected.
    Parameters: 
               Spark DataFrame
    Arguments: 
               Domestic Flights PySpark DataFrame
    Returns:
               A PySpark Dataframe with the Number of flights which arrived earlier than expected.
    """
    
    # Creating a Temporary View for SQL Query
    dataframe.createOrReplaceTempView("domestic_flight_view")

    sql_query_1 = f"""
                      WITH TOTAL_FLIGHT_CTE AS
                      (SELECT
                           COUNT (*) AS `TOTAL FLIGHT`
                       FROM
                         domestic_flight_view),

                      EARLY_ARRIVAL_CTE AS
                      (SELECT
                          COUNT (*) AS `EARLY ARRIVAL`
                      FROM
                          domestic_flight_view
                      WHERE
                          ARR_DELAY < 0)

                      SELECT
                          CONCAT (`EARLY ARRIVAL`, " Flights") AS `EARLY ARRIVALS`,
                          CONCAT (`TOTAL FLIGHT`, " Flights") AS `TOTAL FLIGHTS`,
                          CONCAT (ROUND ((`EARLY ARRIVAL` / `TOTAL FLIGHT`) * 100, 2), " %") AS `PERCENTAGE OF EARLY ARRIVALS`
                      FROM
                          TOTAL_FLIGHT_CTE, EARLY_ARRIVAL_CTE
                   """

    return spark.sql(sql_query_1)
    

In [9]:
print ("The Number of Flights that arrived earlier than expected:-\n")
early_arrivals(domestic_flight_df).show()


The Number of Flights that arrived earlier than expected:-

+--------------+--------------+----------------------------+
|EARLY ARRIVALS| TOTAL FLIGHTS|PERCENTAGE OF EARLY ARRIVALS|
+--------------+--------------+----------------------------+
|534653 Flights|999994 Flights|                     53.47 %|
+--------------+--------------+----------------------------+



---

#### Determine Typical Departure Time for Long-Distance Flights

In [10]:
def typical_departure_time(dataframe):
    """
    Docstring: 
               Take's a PySpark Dataframe as input.
               Determines the Typical Departure Time for flights over 2000 Miles.
               Return a PySpark Dataframe with the Average Departure Time for flights over 2000 Miles.
    Parameters: 
               Spark DataFrame
    Arguments: 
               Domestic Flights PySpark DataFrame
    Returns:
               A PySpark Dataframe with the Average Departure Time for flights over 2000 Miles.
    """
    
    # Creating a Temporary View for SQL Query
    dataframe.createOrReplaceTempView("domestic_flight_view")

    sql_query_1 = f"""
                      SELECT
                          ROUND (AVG(DEP_TIME), 2) AS `TYPICAL DEPARTURE TIME (RAW)`,
                          CONCAT (FLOOR (AVG(DEP_TIME)), " HOUR") AS `TYPICAL DEPARTURE TIME (HOUR)`,
                          CONCAT (ROUND (MOD (AVG(DEP_TIME) * 60, 60), 0), " MINUTES") AS `TYPICAL DEPARTURE TIME (MINUTE)`,
                          
                          CASE WHEN
                              FLOOR (AVG(DEP_TIME)) >= 12 THEN 
                                  CONCAT (FLOOR (AVG(DEP_TIME)), " PM ", ROUND (MOD (AVG(DEP_TIME) * 60, 60), 0), " MINUTES") 
                              ELSE
                                  CONCAT (FLOOR (AVG(DEP_TIME)), " AM ", ROUND (MOD (AVG(DEP_TIME) * 60, 60), 0), " MINUTES")       
                          END AS `TYPICAL DEPARTURE TIME`
                          
                      FROM
                          domestic_flight_view  
                      WHERE
                           DISTANCE > 2000
                   """
    
    return spark.sql(sql_query_1)
    

In [11]:
print ("The Typical Departure Time of Flights with over 2000 Miles is as follows:-\n")
typical_departure_time(domestic_flight_df).show()


The Typical Departure Time of Flights with over 2000 Miles is as follows:-

+----------------------------+-----------------------------+-------------------------------+----------------------+
|TYPICAL DEPARTURE TIME (RAW)|TYPICAL DEPARTURE TIME (HOUR)|TYPICAL DEPARTURE TIME (MINUTE)|TYPICAL DEPARTURE TIME|
+----------------------------+-----------------------------+-------------------------------+----------------------+
|                       13.97|                      13 HOUR|                   58.0 MINUTES|    13 PM 58.0 MINUTES|
+----------------------------+-----------------------------+-------------------------------+----------------------+



---

#### Proportion of Flights with Arrival Delays > 60 Minutes

In [12]:
def proportion_of_flights(dataframe):
    """
    Docstring: 
               Take's a PySpark Dataframe as input.
               Flights with Arrival Delay Longer than 60 Minutes / Total Number of Flights * 100
               Return a PySpark Dataframe with the proportion of flights that have Arrival Delays longer than 60 Minutes.
    Parameters: 
               Spark DataFrame
    Arguments: 
               Domestic Flights PySpark DataFrame
    Returns:
               A PySpark Dataframe with the proportion of flights that have Arrival Delays longer than 60 Minutes.
    """
    
    # Creating a Temporary View for SQL Query
    dataframe.createOrReplaceTempView("domestic_flight_view")

    sql_query_1 = f"""
                      WITH TOTAL_FLIGHTS_CTE AS
                      (SELECT
                           COUNT (*) AS TOTAL_FLIGHTS
                       FROM
                           domestic_flight_view),

                      FLIGHT_DELAY_CTE AS
                      (SELECT
                           COUNT (*) AS FLIGHTS_WITH_DELAY
                       FROM
                           domestic_flight_view
                       WHERE
                           ARR_DELAY > 60)

                      SELECT
                          CONCAT (ROUND((F.FLIGHTS_WITH_DELAY / T.TOTAL_FLIGHTS) * 100, 2), " %") AS `PROPORTION OF FLIGHTS WITH DELAY LONGER THAN SIXTY MINUTES`
                      FROM
                          TOTAL_FLIGHTS_CTE AS T,
                          FLIGHT_DELAY_CTE AS F  
                   """

    return spark.sql(sql_query_1)
    

In [13]:
print (f"The Proportion of Flights with Delay longer than 60 minutues : {proportion_of_flights(domestic_flight_df).collect()[0][0]}\n")
proportion_of_flights(domestic_flight_df).show()


The Proportion of Flights with Delay longer than 60 minutues : 5.31 %

+----------------------------------------------------------+
|PROPORTION OF FLIGHTS WITH DELAY LONGER THAN SIXTY MINUTES|
+----------------------------------------------------------+
|                                                    5.31 %|
+----------------------------------------------------------+



---

#### Average Airtime for Flights Departing Before 9:00 AM

In [14]:
def average_air_time(dataframe):
    """
    Docstring: 
               Take's a PySpark Dataframe as input.
               Finds the Average Airtime (in minutes) for flights that left before 9:00AM.
               Return a PySpark Dataframe with the Average Airtime (in minutes) for flights that left before 9:00AM.
    Parameters: 
               Spark DataFrame
    Arguments: 
               Domestic Flights PySpark DataFrame
    Returns:
               A PySpark Dataframe with the Average Airtime for flights that left before 9:00AM.
    """
    
    # Creating a Temporary View for SQL Query
    dataframe.createOrReplaceTempView("domestic_flight_view")

    sql_query_1 = f"""
                      WITH AVERAGE_AIRTIME_CTE  
                      (SELECT
                          ROUND (AVG(AIR_TIME), 2) AS `AVERAGE AIR TIME (IN RAW MINUTES)`
                      FROM
                          domestic_flight_view
                      WHERE
                          DEP_TIME < 9)

                      SELECT
                          `AVERAGE AIR TIME (IN RAW MINUTES)`,
                          CONCAT (FLOOR (`AVERAGE AIR TIME (IN RAW MINUTES)` / 60), " HOURS") AS `AVERAGE AIR TIME (IN HOURS)`,
                          CONCAT (ROUND (MOD (`AVERAGE AIR TIME (IN RAW MINUTES)`, 60), 0), " MINUTES") AS `AVERAGE AIR TIME (IN MINUTES)`,
                          CONCAT
                              (FLOOR (`AVERAGE AIR TIME (IN RAW MINUTES)` / 60), " HOURS ",
                              ROUND (MOD (`AVERAGE AIR TIME (IN RAW MINUTES)`, 60), 0), " MINUTES")
                                                                                        AS `AVERAGE AIR TIME`
                      FROM
                          AVERAGE_AIRTIME_CTE
                   """

    return spark.sql(sql_query_1)
    

In [15]:
print ("The Average Airtime for flights that left before 9:00 AM is as follows:-\n")
average_air_time(domestic_flight_df).show()


The Average Airtime for flights that left before 9:00 AM is as follows:-

+---------------------------------+---------------------------+-----------------------------+--------------------+
|AVERAGE AIR TIME (IN RAW MINUTES)|AVERAGE AIR TIME (IN HOURS)|AVERAGE AIR TIME (IN MINUTES)|    AVERAGE AIR TIME|
+---------------------------------+---------------------------+-----------------------------+--------------------+
|                           111.36|                    1 HOURS|                 51.0 MINUTES|1 HOURS 51.0 MINUTES|
+---------------------------------+---------------------------+-----------------------------+--------------------+



---

#### Maximum Arrival Delay for Flights With No Departure Delay

In [16]:
def maximum_arrival_delay(dataframe):
    """
    Docstring: 
               Take's a PySpark Dataframe as input.
               Determins the Maximum Arrival Delay for flights that did not experience a delay upon departure.
               Return a PySpark Dataframe with the Maximum Arrival Delay for flights that did not experience a delay upon departure.
    Parameters: 
               Spark DataFrame
    Arguments: 
               Domestic Flights PySpark DataFrame
    Returns:
               A PySpark Dataframe with the Maximum Arrival Delay for flights that did not experience a delay upon departure.
    """
    
    # Creating a Temporary View for SQL Query
    dataframe.createOrReplaceTempView("domestic_flight_view")

    sql_query_1 = f"""
                      WITH MAXIMUM_ARRIVAL_DELAY_CTE AS
                      (SELECT 
                          MAX(ARR_DELAY) AS `MAXIMUM DELAY (IN RAW MINUTES)`
                      FROM
                          domestic_flight_view
                      WHERE
                          DEP_DELAY <= 0)
                          
                      SELECT
                          `MAXIMUM DELAY (IN RAW MINUTES)`,
                          CONCAT (FLOOR (`MAXIMUM DELAY (IN RAW MINUTES)` / 60), ' HOURS') AS `MAXIMUM DELAY (IN HOURS)`,
                          CONCAT (ROUND (MOD (`MAXIMUM DELAY (IN RAW MINUTES)`, 60), 0), ' MINUTES') AS `MAXIMUM DELAY (IN MINUTES)`,
                          CONCAT
                              (FLOOR (`MAXIMUM DELAY (IN RAW MINUTES)` / 60), ' HOURS ',
                              ROUND (MOD (`MAXIMUM DELAY (IN RAW MINUTES)`, 60), 0), ' MINUTES')
                                                                                        AS `MAXIMUM DELAY`
                      FROM
                          MAXIMUM_ARRIVAL_DELAY_CTE 
                   """

    return spark.sql(sql_query_1)
    

In [17]:
print ("The Maximum Arrival Delay for flights that did not experience a delay upon departure is:-\n")
maximum_arrival_delay(domestic_flight_df).show()


The Maximum Arrival Delay for flights that did not experience a delay upon departure is:-

+------------------------------+------------------------+--------------------------+-------------------+
|MAXIMUM DELAY (IN RAW MINUTES)|MAXIMUM DELAY (IN HOURS)|MAXIMUM DELAY (IN MINUTES)|      MAXIMUM DELAY|
+------------------------------+------------------------+--------------------------+-------------------+
|                           701|                11 HOURS|                41 MINUTES|11 HOURS 41 MINUTES|
+------------------------------+------------------------+--------------------------+-------------------+



---

### Conclusions

- **`5,34,653`** Flights arrived earlier than expected. **`53.47%`** percentage of early arrivals.
- The typical departure time for flights over 2000 miles is **`1:58 PM (Noon)`**.
- The Proportion of Flights with Delay longer than 60 minutues : **`5.31 %`**
- The average airtime for flights that left earlier than 9:00 am : **`1 HOUR 51.0 MINUTES`**
- The Maximum Arrival Delay for flights that did not experience a delay upon departure : **`11 HOURS 41 MINUTES`**

---

### Author : Russel Anthony Reynold Chandanshiv

---